In [57]:
# Ячейка 2: подключение библиотек и патч для asyncio в Jupyter
import nest_asyncio
nest_asyncio.apply()  # позволяет запускать вложенные циклы asyncio в ноутбуке

import asyncio
from datetime import datetime
import json

from bs4 import BeautifulSoup
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig
from crawl4ai.content_scraping_strategy import LXMLWebScrapingStrategy
from crawl4ai.deep_crawling import BFSDeepCrawlStrategy
from crawl4ai.deep_crawling.filters import FilterChain, URLPatternFilter

In [59]:
# Ячейка 3: универсальная функция парсинга одного документа
import re
from bs4 import BeautifulSoup
from datetime import datetime

def parse_decision(html: str) -> dict:
    soup = BeautifulSoup(html, "lxml")
    
    # 1) Заголовок / номер дела
    h1 = soup.find("h1")
    title = h1.get_text(strip=True) if h1 else ""
    m = re.search(r"№\s*([\d/]+)", title)
    case_number = m.group(1) if m else title or "—"
    
    # 2) Дата
    time_tag = soup.find("time")
    if time_tag:
        date = time_tag.get_text(strip=True)
    else:
        m2 = re.search(r"\b\d{2}\.\d{2}\.\d{4}\b", html)
        date = m2.group(0) if m2 else "—"
    # Попробуем привести к ISO
    try:
        date = datetime.strptime(date, "%d.%m.%Y").date().isoformat()
    except Exception:
        pass
    
    # 3) Основной блок текста
    # Попробуем найти некий контейнер с контентом
    body_container = (
        soup.find("div", class_="doc-content") or 
        soup.find("div", class_="b-article") or
        soup.find("div", class_="content")
    )
    if body_container:
        paras = [p.get_text(strip=True) for p in body_container.find_all("p")]
    else:
        # Фолбэк — все параграфы на странице
        paras = [p.get_text(strip=True) for p in soup.find_all("p")]
    
    text = "\n\n".join(filter(None, paras)) or "—"
    
    # 4) Участники (если есть специфичный список, иначе пусто)
    participants = []
    for li in soup.select("ul.participants li"):
        t = li.get_text(strip=True)
        if t:
            participants.append(t)
    
    return {
        "case_number": case_number,
        "date": date,
        "participants": participants,
        "text": text
    }

In [60]:
# Ячейка 4: функция форматирования в Markdown
def format_markdown(entry: dict) -> str:
    """
    Возвращает в Markdown только текст решения с разделителем.
    """
    return entry["text"] + "\n\n---\n\n"

In [61]:
# Ячейка 5: асинхронная функция обхода и сохранения в Markdown
async def crawl_and_save_md(start_url: str, output_path: str):
    # Разрешаем переход только на страницы individual document
    url_filter = URLPatternFilter(patterns=["*sudrf.cntd.ru/document/*"])
    
    strategy = BFSDeepCrawlStrategy(
        max_depth=1,               # 0 — список, 1 — документы
        include_external=False,
        filter_chain=FilterChain([url_filter]),
        max_pages=1000              # ограничиваем до 1000 документов
    )
    config = CrawlerRunConfig(
        deep_crawl_strategy=strategy,
        scraping_strategy=LXMLWebScrapingStrategy(),
        stream=True,
        verbose=False
    )

    with open(output_path, "w", encoding="utf-8") as md_file:
        async with AsyncWebCrawler() as crawler:
            async for result in await crawler.arun(start_url, config=config):
                depth = result.metadata.get("depth", 0)
                # пропускаем стартовую страницу списка
                if depth == 0 or not result.success:
                    continue

                parsed = parse_decision(result.html)
                # записываем только текст
                md_file.write(format_markdown(parsed))
                print(f"✔ {result.url} сохранён ({depth=})")

    print(f"\nГотово: сохранено до 1000 документов в «{output_path}»")

In [62]:
# Ячейка 6: обход сразу двух списков и сохранение в Markdown
# Ячейка 6: обход двух списков с новыми именами файлов
DOCUMENTS = {
    "901812182": "court_decisions_obsh_urisdikcii.md",
    "780500005": "court_decisions_arbitrag.md"
}

for doc_id, out_file in DOCUMENTS.items():
    start_url = f"https://sudrf.cntd.ru/documents?id={doc_id}"
    print(f"\n=== Старт краулинга для {start_url} ===")
    await crawl_and_save_md(start_url, out_file)


=== Старт краулинга для https://sudrf.cntd.ru/documents?id=901812182 ===


[INIT].... → Crawl4AI 0.6.3 

✔ https://sudrf.cntd.ru/document/1313022993 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125371 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125169 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125332 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125348 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125282 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125358 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125104 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125333 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125224 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125296 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125356 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125306 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125176 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125280 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313125389 сохранён (d

[INIT].... → Crawl4AI 0.6.3 

✔ https://sudrf.cntd.ru/document/1313002290 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002341 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002347 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002330 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002355 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002348 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002313 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002356 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002301 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002311 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002297 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002305 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002293 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002302 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002294 сохранён (depth=1)
✔ https://sudrf.cntd.ru/document/1313002306 сохранён (d

In [63]:
from pathlib import Path

def merge_markdown_files(input_paths, output_path, separator='\n\n'):
    """
    Объединяет несколько Markdown-файлов в один.
    
    :param input_paths: список путей к исходным файлам (Path или str)
    :param output_path: путь к результирующему файлу (Path или str)
    :param separator: строка-разделитель между содержимым файлов
    """
    output_path = Path(output_path)
    contents = []

    for p in input_paths:
        p = Path(p)
        if not p.exists():
            raise FileNotFoundError(f"Файл не найден: {p}")
        text = p.read_text(encoding='utf-8')
        contents.append(text)

    # Объединяем с указанным разделителем
    merged_text = separator.join(contents)

    # Сохраняем в выходной файл
    output_path.write_text(merged_text, encoding='utf-8')
    print(f"Успешно создан файл: {output_path}")

if __name__ == "__main__":
    files_to_merge = [
        "court_decisions_obsh_urisdikcii.md",
        "court_decisions_arbitrag.md"
    ]
    output_file = "court_decisions_combined.md"
    merge_markdown_files(files_to_merge, output_file)

Успешно создан файл: court_decisions_combined.md
